In [ ]:
from kaggle_environments import make

from pypad.solvers.network_torch import PytorchNeuralNetwork
from pypad.solvers.alpha_zero import AlphaZero
from pypad.solvers.mcts import MctsSolver
from pypad.games import ConnectX, TicTacToe
from pypad.views import policies

# Visualising Model predictions for TicTacToe

In [ ]:
tictactoe = TicTacToe()
tictactoe_state = tictactoe.initial_state("3,4,2")
tictactoe_state

In [ ]:
tictactoe_network = PytorchNeuralNetwork.create(tictactoe, '../weights')
alpha_zero_ttt = AlphaZero(tictactoe_network)

policy, value = alpha_zero_ttt.policy(tictactoe_state, num_mcts_sims=100, is_raw_policy=True)
tictactoe_state.show_policy(policy, value)

# Visualising Model predictions for ConnectX

In [ ]:
connectx = ConnectX()
connectx_state = connectx.initial_state([4,4,4,4,4,3,2,1,3,3,3,5,5,6,5,5,5,6,7,7,7,7])
connectx_state

In [ ]:
connectx_network = PytorchNeuralNetwork.create(connectx, '../weights', 48)
alpha_zero_connectx = AlphaZero(connectx_network)

policy, value = alpha_zero_connectx.policy(connectx_state, num_mcts_sims=1_000, is_raw_policy=True)
connectx_state.show_policy(policy, value)

# Alpha Zero vs Classical Monte Carlo Tree Search

In [ ]:
player_1000 = MctsSolver(1_000)
agent_1000 = connectx.create_agent(player_1000)

alpha_zero_agent = connectx.create_agent(alpha_zero_connectx)

# Run on the Kaggle environment
env = make(connectx.name, debug=True)
env.run([agent_1000, alpha_zero_agent])
env.render(mode="ipython")